## imports

In [1]:
import pandas
import numpy
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
import mysql.connector
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
import joblib
import datetime
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.vector_ar.var_model import VAR

%matplotlib qt

## Database Connection and data Fetching

In [2]:
with open('../config.env') as f:
    credentials=f.read()
credentials=credentials.split(" ")
con = mysql.connector.connect(username=credentials[0],password=credentials[1],host=credentials[2],port=credentials[3],database=credentials[4])
query=con.cursor()

In [3]:
sql="SELECT * FROM total_india_cases"
query.execute(sql)
result=query.fetchall()
column=[columns[0] for columns in query.description]
india_data=pandas.DataFrame(result,columns=column)

In [4]:
india_data

,index_no,date,ordinal_date,total_confirmed,total_active,total_recovered,total_deaths,total_tested,total_vaccinated1,total_vaccinated2,...,delta7_confirmed,delta7_active,delta7_recovered,delta7_deaths,delta7_tested,delta7_vaccinated1,delta7_vaccinated2,total_other,delta_other,delta7_other
0,1,2020-01-30,737454,1,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,2,2020-02-02,737457,2,2,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
2,3,2020-02-03,737458,3,3,0,0,0,0,0,...,3,3,0,0,0,0,0,0,0,0
3,4,2020-02-14,737469,3,0,3,0,0,0,0,...,0,-3,3,0,0,0,0,0,0,0
4,5,2020-03-02,737486,5,2,3,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,582,2021-09-29,738062,33738217,269744,33007340,448094,568956439,646473741,236996837,...,176115,-25088,199126,2013,10589426,27960741,21519788,13039,9,64
581,583,2021-09-30,738063,33765502,268540,33035544,448372,570477338,650229045,239978962,...,171989,-24862,194820,1973,10544629,27326652,21363329,13046,7,58
582,584,2021-10-01,738064,33789420,266757,33061004,448605,571994990,654331744,243149810,...,166342,-27862,192231,1915,10333607,27156870,21395524,13054,8,58
583,585,2021-10-02,738065,33812609,263763,33086942,448847,573260724,658277760,246897588,...,161362,-32730,192148,1898,10017479,27446675,21647146,13057,3,46


In [5]:
sql="SELECT * FROM total_state_cases WHERE state_name='MH'"
query.execute(sql)
result=query.fetchall()
column=[columns[0] for columns in query.description]
maharashtra_data=pandas.DataFrame(result,columns=column)

In [6]:
maharashtra_data

,index_no,date,ordinal_date,state_name,total_confirmed,total_active,total_recovered,total_deaths,total_tested,total_vaccinated1,...,delta7_confirmed,delta7_active,delta7_recovered,delta7_deaths,delta7_tested,delta7_vaccinated1,delta7_vaccinated2,total_other,delta_other,delta7_other
0,52,2020-03-09,737493,MH,2,2,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
1,64,2020-03-10,737494,MH,5,5,0,0,0,0,...,5,5,0,0,0,0,0,0,0,0
2,76,2020-03-11,737495,MH,11,11,0,0,0,0,...,11,11,0,0,0,0,0,0,0,0
3,89,2020-03-12,737496,MH,14,14,0,0,0,0,...,14,14,0,0,0,0,0,0,0,0
4,102,2020-03-13,737497,MH,17,17,0,0,0,0,...,17,17,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,19917,2021-09-29,738062,MH,6547793,36675,6368530,139011,58884819,57091306,...,16556,-3309,19501,347,1408677,2729841,2110890,3577,4,17
570,19953,2021-09-30,738063,MH,6550856,36484,6371728,139067,58739974,57633028,...,16299,-2707,18649,342,1093459,2820452,2153323,3577,0,15
571,19989,2021-10-01,738064,MH,6553961,36371,6374892,139117,58910564,58110270,...,16118,-2120,17880,341,1091179,3013433,2263721,3581,4,17
572,20025,2021-10-02,738065,MH,6556657,35955,6377954,139166,59074660,58553667,...,15538,-2029,17219,332,1082650,3140902,2288397,3582,1,16


In [7]:
sql="SELECT * FROM total_district_cases WHERE district_name='Raigad'"
query.execute(sql)
result=query.fetchall()
column=[columns[0] for columns in query.description]
raigad_data=pandas.DataFrame(result,columns=column)

In [8]:
raigad_data

,index_no,date,ordinal_date,state_name,district_name,total_confirmed,total_active,total_recovered,total_deaths,delta_confirmed,...,delta7_deaths,total_vaccinated1,total_vaccinated2,delta_vaccinated1,delta_vaccinated2,delta7_vaccinated1,delta7_vaccinated2,total_other,delta_other,delta7_other
0,19918,2020-04-26,737541,MH,Raigad,57,36,20,1,0,...,0,0,0,0,0,0,0,0,0,0
1,20334,2020-04-27,737542,MH,Raigad,61,39,21,1,4,...,0,0,0,0,0,0,0,0,0,0
2,20759,2020-04-28,737543,MH,Raigad,66,44,21,1,5,...,0,0,0,0,0,0,0,0,0,0
3,21189,2020-04-29,737544,MH,Raigad,69,44,23,2,3,...,1,0,0,0,0,0,0,0,0,0
4,21623,2020-04-30,737545,MH,Raigad,71,44,24,3,2,...,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,379646,2021-09-29,738062,MH,Raigad,192684,1070,187098,4509,157,...,6,1497575,596105,45182,16086,112372,59902,7,0,0
522,380393,2021-09-30,738063,MH,Raigad,192810,1077,187216,4510,126,...,6,1523225,605831,25650,9726,131856,64441,7,0,0
523,381140,2021-10-01,738064,MH,Raigad,192930,977,187435,4511,120,...,7,1551617,615525,28392,9694,157100,70365,7,0,0
524,381887,2021-10-02,738065,MH,Raigad,193070,836,187715,4512,140,...,6,1567578,625129,15961,9604,133527,60062,7,0,0


## function for traintest_splitting and next 7 days ordinal_date creating

In [9]:
def splitting(data,training_ratio):
    length=len(data)
    traing_indexes=int(length*training_ratio)
    training_set=data[:traing_indexes]
    testing_set=data[traing_indexes:]
    return training_set,testing_set

In [106]:
def returnNext7days():
    date=738066
    date_range=[]
    l=[]
    for i in range(1,8):
        l.append([date+i])
        date_range.append(datetime.date.fromordinal(date+i))
    return l,date_range
predictingdate,daterange=returnNext7days()


## Train Test Splitting

In [11]:
# maindata=india_data
# maindata=maharashtra_data
maindata=raigad_data

In [12]:
maindata_labels=maindata['delta_confirmed']

In [13]:
maindata

,index_no,date,ordinal_date,state_name,district_name,total_confirmed,total_active,total_recovered,total_deaths,delta_confirmed,...,delta7_deaths,total_vaccinated1,total_vaccinated2,delta_vaccinated1,delta_vaccinated2,delta7_vaccinated1,delta7_vaccinated2,total_other,delta_other,delta7_other
0,19918,2020-04-26,737541,MH,Raigad,57,36,20,1,0,...,0,0,0,0,0,0,0,0,0,0
1,20334,2020-04-27,737542,MH,Raigad,61,39,21,1,4,...,0,0,0,0,0,0,0,0,0,0
2,20759,2020-04-28,737543,MH,Raigad,66,44,21,1,5,...,0,0,0,0,0,0,0,0,0,0
3,21189,2020-04-29,737544,MH,Raigad,69,44,23,2,3,...,1,0,0,0,0,0,0,0,0,0
4,21623,2020-04-30,737545,MH,Raigad,71,44,24,3,2,...,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,379646,2021-09-29,738062,MH,Raigad,192684,1070,187098,4509,157,...,6,1497575,596105,45182,16086,112372,59902,7,0,0
522,380393,2021-09-30,738063,MH,Raigad,192810,1077,187216,4510,126,...,6,1523225,605831,25650,9726,131856,64441,7,0,0
523,381140,2021-10-01,738064,MH,Raigad,192930,977,187435,4511,120,...,7,1551617,615525,28392,9694,157100,70365,7,0,0
524,381887,2021-10-02,738065,MH,Raigad,193070,836,187715,4512,140,...,6,1567578,625129,15961,9604,133527,60062,7,0,0


In [14]:
train_data,test_data=splitting(maindata,0.98)

In [15]:
train_labels=train_data['delta_confirmed']
# train_labels=train_data['total_confirmed']

In [16]:
# india_train_labels

In [17]:
train_features=train_data[['ordinal_date']]
# train_fetures=train_data[['ordinal_date','delta_tested']]

In [18]:
train_features

,ordinal_date
0,737541
1,737542
2,737543
3,737544
4,737545
...,...
510,738051
511,738052
512,738053
513,738054


In [19]:
# train_labels

In [20]:
test_labels=test_data['delta_confirmed']
# test_labels=test_data['total_confirmed']

In [21]:
# test_labels

In [22]:
test_features=test_data[['ordinal_date']]
# test_features=test_data[['ordinal_date','delta_tested']]

In [23]:
# test_features

In [24]:
# print(predictingdate)
# print(daterange)

## Model Implementation 
1. Linear Regression
2. Random Forest Regression
3. Decision Tree Regression
4. Extra Tree Regression

In [28]:
linear_regression_model=LinearRegression()
random_forest_model=RandomForestRegressor()
decision_tree_model=DecisionTreeRegressor()
extra_tree_model=ExtraTreeRegressor()

#### Linear regression

In [29]:
linear_regression_model.fit(train_features,train_labels)

LinearRegression()

In [30]:
linear_regression_model_train_predicted=linear_regression_model.predict(train_features)

In [31]:
# linear_regression_model_train_predicted

In [32]:
linear_regression_model_test_predicted=linear_regression_model.predict(test_features)

In [33]:
# linear_regression_model_test_predicted

In [34]:
linear_regression_model_next_7_days_predicted=linear_regression_model.predict(predictingdate)

In [147]:
# linear_regression_model_next_7_days_predicted

In [148]:
pyplot.figure('linear regression')
line_of_regression=numpy.concatenate((linear_regression_model_train_predicted,linear_regression_model_test_predicted))
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(maindata['date'],line_of_regression,label='Line of Regression')
pyplot.plot(train_data['date'],linear_regression_model_train_predicted,label='Train Data')
pyplot.plot(test_data['date'],linear_regression_model_test_predicted,label='Test Data')
pyplot.plot(daterange,linear_regression_model_next_7_days_predicted,label='Future Prediction')
pyplot.legend()
pyplot.show()

#### Ramdom Forest Regression regression

In [35]:
random_forest_model.fit(train_features,train_labels)

RandomForestRegressor()

In [36]:
random_forest_model_train_predicted=random_forest_model.predict(train_features)

In [37]:
# random_forest_model_train_predicted

In [38]:
random_forest_model_test_predicted=random_forest_model.predict(test_features)

In [39]:
# random_forest_model_test_predicted

In [40]:
random_forest_model_next_7_days_predicted=random_forest_model.predict(predictingdate)

In [155]:
# random_forest_model_next_7_days_predicted

In [156]:
pyplot.figure('random forest regression')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],random_forest_model_train_predicted,label='Train Data')
pyplot.plot(test_data['date'],random_forest_model_test_predicted,label='Test Data')
pyplot.plot(daterange,random_forest_model_next_7_days_predicted,label='Future Prediction')
pyplot.legend()
pyplot.show()

#### Decision Tree regression

In [41]:
decision_tree_model.fit(train_features,train_labels)

DecisionTreeRegressor()

In [42]:
decision_tree_model_train_predicted=decision_tree_model.predict(train_features)

In [43]:
# decision_tree_model_train_predicted

In [44]:
decision_tree_model_test_predicted=decision_tree_model.predict(test_features)

In [45]:
# decision_tree_model_test_predicted

In [46]:
decision_tree_model_next_y_days_predicted=decision_tree_model.predict(predictingdate)

In [163]:
# decision_tree_model_next_y_days_predicted

In [49]:
pyplot.figure('Decision tree regression')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],decision_tree_model_train_predicted,label='Train Data')
pyplot.plot(test_data['date'],decision_tree_model_test_predicted,label='Test Data')
pyplot.plot(daterange,decision_tree_model_next_y_days_predicted,label='Future Prediction')
pyplot.legend()
pyplot.show()

#### Extra Tree regression

In [47]:
extra_tree_model.fit(train_features,train_labels)

ExtraTreeRegressor()

In [48]:
extra_tree_model_train_predicted=extra_tree_model.predict(train_features)

In [49]:
# extra_tree_model_train_predicted

In [50]:
extra_tree_model_test_predicted=extra_tree_model.predict(test_features)

In [51]:
# extra_tree_model_test_predicted

In [52]:
extra_tree_model_next_7_days_predicted=extra_tree_model.predict(predictingdate)

In [56]:
# extra_tree_model_next_7_days_predicted

In [57]:
pyplot.figure('Extra tree regression')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],extra_tree_model_train_predicted,label='Train Data')
pyplot.plot(test_data['date'],extra_tree_model_test_predicted,label='Test Data')
pyplot.plot(daterange,extra_tree_model_next_7_days_predicted,label='Future Prediction')
pyplot.legend()
pyplot.show()

## Evaluation
    model
    train : mean_absolute_error     root_mean_squared_error
    test :  mean_absolute_error     root_mean_squared_error

In [53]:
print('linear regrassion')
print(mean_absolute_error(linear_regression_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(linear_regression_model_train_predicted,train_labels)))
print(mean_absolute_error(linear_regression_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(linear_regression_model_test_predicted,test_labels)))

linear regrassion
283.0350918400663 367.4276928101792
411.59635891780175 412.09389937938204


In [54]:
print('random forest regrassion')
print(mean_absolute_error(random_forest_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(random_forest_model_train_predicted,train_labels)))
print(mean_absolute_error(random_forest_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(random_forest_model_test_predicted,test_labels)))

random forest regrassion
25.455611650485437 47.57074254223301
16.747272727272726 20.138006581856832


In [55]:
print('Decision Tree regression')
print(mean_absolute_error(decision_tree_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(decision_tree_model_train_predicted,train_labels)))
print(mean_absolute_error(decision_tree_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(decision_tree_model_test_predicted,test_labels)))

Decision Tree regression
0.0 0.0
17.545454545454547 22.98813923427787


In [56]:
print('Extra Tree Regression')
print(mean_absolute_error(extra_tree_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(extra_tree_model_train_predicted,train_labels)))
print(mean_absolute_error(extra_tree_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(extra_tree_model_test_predicted,test_labels)))

Extra Tree Regression
0.0 0.0
17.545454545454547 22.98813923427787


## Polynomial Regression

In [62]:
# pyplot.scatter(train_features,train_labels)

In [57]:
def polynomial_regrassion_model(degree_of_equation=1):
    polynomial=PolynomialFeatures(degree=degree_of_equation)
    polynomial_train_data=polynomial.fit_transform(train_features.to_numpy())
    polynomial_test_data=polynomial.fit_transform(test_features.to_numpy())
    polynomial_future_test_data=polynomial.fit_transform(predictingdate)
    polynomial_regression=LinearRegression()
    polynomial_regression.fit(polynomial_train_data,train_labels)
    polynomial_regression_model_train_predicted=polynomial_regression.predict(polynomial_train_data)
    polynomial_regression_model_test_predicted=polynomial_regression.predict(polynomial_test_data)
    polynomial_regression_model_nest_7_days_predicted=polynomial_regression.predict(polynomial_future_test_data)
#     print('Train Data Prediction')
#     print(polynomial_regression_model_train_predicted)
#     print('\n\n\nTest Data Prediction')
#     print(polynomial_regression_model_test_predicted)
#     print('\n\n\nFuture Data Prediction')
#     print(polynomial_regression_model_nest_7_days_predicted)
#     print('\n\n\n')
    print('{} degree Polynomial regression'.format(degree_of_equation))
    print(mean_absolute_error(polynomial_regression_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(polynomial_regression_model_train_predicted,train_labels)))
    print(mean_absolute_error(polynomial_regression_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(polynomial_regression_model_test_predicted,test_labels)))
    pyplot.figure('{} degree Polynomial regression'.format(degree_of_equation))
    pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
    pyplot.plot(train_data['date'],polynomial_regression_model_train_predicted,label='Train Data')
    pyplot.plot(test_data['date'],polynomial_regression_model_test_predicted,label='Test Data')
    pyplot.plot(daterange,polynomial_regression_model_nest_7_days_predicted,label='Future Prediction')
    pyplot.legend()
    pyplot.show()

In [58]:
# polynomial_regrassion_model()

In [59]:
polynomial_regrassion_model(2)

2 degree Polynomial regression
276.3802224159241 361.48684801385497
254.8866225155917 255.54079606195526


In [60]:
polynomial_regrassion_model(3)

3 degree Polynomial regression
276.379019039117 361.48503288328817
254.83921189741656 255.4934679097837


In [61]:
polynomial_regrassion_model(4)

4 degree Polynomial regression
276.3778160245673 361.48321684386343
254.79218472675845 255.44652233877613


In [179]:
polynomial_regrassion_model(5)

5 degree Polynomial regression
277.54133528400735 362.38945785826195
287.2486034469171 287.87667181210423


In [62]:
polynomial_regrassion_model(6)

6 degree Polynomial regression
276.37540626441967 361.4795820176879
254.6980803310871 255.35258136262445


# Forcasting Models

In [107]:
# import warnings
# warnings.filterwarnings('ignore')

In [108]:
forcasting_data_features=maindata[['date','delta_confirmed']]

forcasting_train_data_features=train_data[['date','delta_confirmed']]

forcasting_test_data_features=test_data[['date','delta_confirmed']]

In [109]:
forcasting_data_features['date']=pandas.to_datetime(forcasting_data_features['date'])

forcasting_train_data_features['date']=pandas.to_datetime(forcasting_train_data_features['date'])

forcasting_test_data_features['date']=pandas.to_datetime(forcasting_test_data_features['date'])

<ipython-input-109-7a0089d374bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forcasting_data_features['date']=pandas.to_datetime(forcasting_data_features['date'])
<ipython-input-109-7a0089d374bb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forcasting_train_data_features['date']=pandas.to_datetime(forcasting_train_data_features['date'])
<ipython-input-109-7a0089d374bb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [110]:
forcasting_data_features=forcasting_data_features.set_index('date')

forcasting_train_data_features=forcasting_train_data_features.set_index('date')

forcasting_test_data_features=forcasting_test_data_features.set_index('date')

In [111]:
# forcasting_data_features

In [112]:
# forcasting_train_data_features

In [113]:
# forcasting_test_data_features

## ARIMA model

In [114]:
main_arima_model = ARIMA(forcasting_data_features,order=(5,0,3))
train_arima_model = ARIMA(forcasting_train_data_features,order=(5,0,3))

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\Pankaj sing

In [115]:
main_arima_fit_model=main_arima_model.fit()
train_arima_fit_model=train_arima_model.fit()

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum L

In [116]:
main_arima_fit_model_predicted=main_arima_fit_model.predict()
train_arima_fit_model_predicted=train_arima_fit_model.predict()
test_arima_predicted=train_arima_fit_model.forecast(steps=len(forcasting_test_data_features))

In [117]:
arima_model_forcast=main_arima_fit_model.forecast(steps=7)

In [118]:
# main_arima_fit_model_predicted
# train_arima_fit_model_predicted
# test_arima_predicted
arima_model_forcast

2021-10-04    125.689693
2021-10-05    137.998759
2021-10-06    141.005693
2021-10-07    134.614140
2021-10-08    147.632227
2021-10-09    140.636921
2021-10-10    148.903543
Freq: D, Name: predicted_mean, dtype: float64

In [141]:
# pyplot.figure('ARIMA model Forcasting')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],train_arima_fit_model_predicted,label='Train Data')
pyplot.plot(test_data['date'],test_arima_predicted,label='Test Data')
pyplot.plot(daterange,arima_model_forcast,label='ARIMA Future Prediction')
pyplot.legend()
pyplot.show()

In [120]:
print(mean_absolute_error(train_arima_fit_model_predicted,train_labels),numpy.sqrt(mean_squared_error(train_arima_fit_model_predicted,train_labels)))
print(mean_absolute_error(test_arima_predicted,test_labels),numpy.sqrt(mean_squared_error(test_arima_predicted,test_labels)))
    

76.02358105865368 130.33387172721095
24.116503191991015 31.404988512543934


In [121]:
train_data['ordinal_date']

0      737541
1      737542
2      737543
3      737544
4      737545
        ...  
510    738051
511    738052
512    738053
513    738054
514    738055
Name: ordinal_date, Length: 515, dtype: int64

In [122]:
try_data=numpy.concatenate((maindata['ordinal_date'].to_numpy(),predictingdate),axis=None)

In [123]:
# test_arima_predicted.to_numpy()

In [124]:
# train_data['ordinal_date'].to_numpy()

In [125]:
try_data[-20:]

array([738054, 738055, 738056, 738057, 738058, 738059, 738060, 738061,
       738062, 738063, 738064, 738065, 738066, 738067, 738068, 738069,
       738070, 738071, 738072, 738073], dtype=int64)

In [126]:
try_labels=numpy.concatenate((maindata_labels,arima_model_forcast))

In [127]:
try_labels[-20:]

array([138.        , 150.        , 164.        , 131.        ,
       149.        , 156.        ,  97.        , 151.        ,
       157.        , 126.        , 120.        , 140.        ,
       122.        , 125.68969345, 137.99875949, 141.00569339,
       134.61414006, 147.63222714, 140.63692054, 148.9035433 ])

In [128]:
random_forest_model.fit(pandas.DataFrame(try_data),try_labels)

RandomForestRegressor()

In [129]:
random_forest_model.predict(pandas.DataFrame(try_data))

array([1.43000000e+00, 3.19000000e+00, 4.46000000e+00, 3.47000000e+00,
       2.49000000e+00, 2.73000000e+00, 3.05000000e+00, 8.09000000e+00,
       2.07800000e+01, 4.31900000e+01, 2.39600000e+01, 2.18000000e+01,
       1.57400000e+01, 1.22600000e+01, 8.80000000e+00, 2.25700000e+01,
       1.83000000e+01, 2.77000000e+01, 2.46300000e+01, 4.69000000e+01,
       3.11400000e+01, 2.94000000e+01, 2.80500000e+01, 3.22900000e+01,
       2.67700000e+01, 2.44000000e+01, 2.54000000e+01, 3.77300000e+01,
       9.14600000e+01, 6.89600000e+01, 5.66100000e+01, 5.28400000e+01,
       4.93200000e+01, 5.27300000e+01, 4.71900000e+01, 5.87800000e+01,
       4.83200000e+01, 5.31700000e+01, 4.22100000e+01, 5.18400000e+01,
       6.05600000e+01, 4.94900000e+01, 3.55500000e+01, 2.70400000e+01,
       4.01600000e+01, 6.26100000e+01, 6.40700000e+01, 7.10300000e+01,
       7.14600000e+01, 7.93100000e+01, 6.72300000e+01, 5.47900000e+01,
       8.56900000e+01, 9.68700000e+01, 1.02650000e+02, 1.25420000e+02,
      

## SARIMA model

In [130]:
main_sarima_model=SARIMAX(forcasting_data_features)
train_sarima_model = ARIMA(forcasting_train_data_features,order=(5,0,3))

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\Pankaj sing

In [131]:
main_sarima_fit_model=main_sarima_model.fit()
train_sarima_fit_model=train_sarima_model.fit()

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [132]:
# sarima_model_fit.predict(590)
main_sarima_fit_model_predicted=main_sarima_fit_model.predict()
train_sarima_fit_model_predicted=train_sarima_fit_model.predict()
test_sarima_predicted=train_sarima_fit_model.forecast(steps=len(forcasting_test_data_features))

In [133]:
sarima_model_forcast=main_sarima_fit_model.forecast(steps=7)

In [144]:
# main_sarima_fit_model_predicted
# train_sarima_fit_model_predicted
# test_sarima_predicted
sarima_model_forcast

2021-10-04    116.816276
2021-10-05    111.852807
2021-10-06    107.100233
2021-10-07    102.549593
2021-10-08     98.192308
2021-10-09     94.020163
2021-10-10     90.025289
Freq: D, Name: predicted_mean, dtype: float64

In [143]:
pyplot.figure('SARIMA model Forcasting')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],train_sarima_fit_model_predicted,label='Train Data')
pyplot.plot(test_data['date'],test_sarima_predicted,label='Test Data')
pyplot.plot(daterange,sarima_model_forcast,label='SARIMA Future Prediction')
pyplot.legend()
pyplot.show()

In [136]:
print(mean_absolute_error(train_sarima_fit_model_predicted,train_labels),numpy.sqrt(mean_squared_error(train_sarima_fit_model_predicted,train_labels)))
print(mean_absolute_error(test_sarima_predicted,test_labels),numpy.sqrt(mean_squared_error(test_sarima_predicted,test_labels)))
    

76.02358105865368 130.33387172721095
24.116503191991015 31.404988512543934


116.81627636131688

## VAR Model

In [93]:
forcasting_data_features_for_VAR=maindata[['date','delta_confirmed','delta_vaccinated1','delta_vaccinated2']]
forcasting_train_data_features_for_VAR=train_data[['date','delta_confirmed','delta_vaccinated1','delta_vaccinated2']]

In [94]:
forcasting_data_features_for_VAR
# forcasting_train_data_features_for_VAR

,date,delta_confirmed,delta_vaccinated1,delta_vaccinated2
0,2020-04-26,0,0,0
1,2020-04-27,4,0,0
2,2020-04-28,5,0,0
3,2020-04-29,3,0,0
4,2020-04-30,2,0,0
...,...,...,...,...
521,2021-09-29,157,45182,16086
522,2021-09-30,126,25650,9726
523,2021-10-01,120,28392,9694
524,2021-10-02,140,15961,9604


In [95]:
forcasting_data_features_for_VAR['date']=pandas.to_datetime(forcasting_data_features_for_VAR['date'])
forcasting_train_data_features_for_VAR['date']=pandas.to_datetime(forcasting_train_data_features_for_VAR['date'])

<ipython-input-95-1b0de2c1aaa6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forcasting_data_features_for_VAR['date']=pandas.to_datetime(forcasting_data_features_for_VAR['date'])
<ipython-input-95-1b0de2c1aaa6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forcasting_train_data_features_for_VAR['date']=pandas.to_datetime(forcasting_train_data_features_for_VAR['date'])


In [96]:
# forcasting_data_features_for_VAR
forcasting_train_data_features_for_VAR

,date,delta_confirmed,delta_vaccinated1,delta_vaccinated2
0,2020-04-26,0,0,0
1,2020-04-27,4,0,0
2,2020-04-28,5,0,0
3,2020-04-29,3,0,0
4,2020-04-30,2,0,0
...,...,...,...,...
510,2021-09-18,145,21601,8403
511,2021-09-19,156,6999,5738
512,2021-09-20,111,13568,6078
513,2021-09-21,138,32266,13366


In [97]:
forcasting_data_features_for_VAR=forcasting_data_features_for_VAR.set_index('date')
forcasting_train_data_features_for_VAR=forcasting_train_data_features_for_VAR.set_index('date')

In [98]:
# forcasting_data_features_for_VAR
forcasting_train_data_features_for_VAR

,delta_confirmed,delta_vaccinated1,delta_vaccinated2
date,,,
2020-04-26,0,0,0
2020-04-27,4,0,0
2020-04-28,5,0,0
2020-04-29,3,0,0
2020-04-30,2,0,0
...,...,...,...
2021-09-18,145,21601,8403
2021-09-19,156,6999,5738
2021-09-20,111,13568,6078


In [99]:
main_var_model=VAR(forcasting_data_features_for_VAR)
main_var_train_model=VAR(forcasting_train_data_features_for_VAR)

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'


In [100]:
main_var_model_fit=main_var_model.fit()
main_var_train_model_fit=main_var_train_model.fit()

In [101]:
# main_var_train_model_fit.plot()

In [102]:
var_model_forcast=main_var_model_fit.forecast(main_var_model.y,steps=7)

<ipython-input-102-2bd438d2e6bb>:1: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  var_model_forcast=main_var_model_fit.forecast(main_var_model.y,steps=7)


In [103]:
var_model_forcast_data=pandas.DataFrame(var_model_forcast)

In [104]:
var_model_forcast_data

,0,1,2
0,136.823286,3893.409058,2142.133839
1,153.840091,3550.610839,1744.256164
2,170.726987,3269.201191,1509.831733
3,186.679600,3079.440216,1373.740039
4,201.475710,2964.286750,1295.590532
5,215.107033,2900.210043,1251.112583
6,227.637314,2868.277332,1226.013062


In [105]:
pyplot.figure('VAR model Forcasting')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(maindata['date'],maindata['delta_vaccinated1'],label='Original Data')
pyplot.plot(daterange,var_model_forcast_data[0],label='VAR Future Confirmed Prediction')
pyplot.plot(daterange,var_model_forcast_data[1],label='VAR Future Vaccinated1 Prediction')

pyplot.legend()
pyplot.show()